# Docker安装运行FastDFS

- 1.获取FastDFS镜像

```shell
#从仓库拉取镜像
sudo docker image pull delron/fastdfs
# 解压教学资料中本地镜像
sudo docker load -i 文件路径/fastdfs_docker.tar
```

- 2.开启tracker容器
我们将 tracker 运行目录映射到宿主机的 /var/fdfs/tracker目录中。

```shell
sudo docker run -dit --name tracker --network=host -v /var/fdfs/tracker:/var/fdfs delron/fastdfs tracker
```

- 3.开启storage容器
TRACKER_SERVER=Tracker的ip地址:22122（Tracker的ip地址不要使用127.0.0.1）
我们将 storage 运行目录映射到宿主机的 /var/fdfs/storage目录中。

```shell
sudo docker run -dti --name storage --network=host -e TRACKER_SERVER=192.168.103.158:22122 -v /var/fdfs/storage:/var/fdfs delron/fastdfs storage
```

- 4.查看宿主机映射路径
 

  * 注意：如果无法重启storage容器，可以删除/var/fdfs/storage/data目录下的fdfs_storaged.pid 文件，然后重新运行storage。
  
- 5.如果想停止tracker服务，可以执行如下命令

```shell
sudo docker container stop tracker
sudo docker container stop storage
```

重新运行tracker，可以执行如下命令

```shell
sudo docker container start storage
sudo docker container start tracker
```

- 6.执行如下命令开启storage服务

```shell
sudo docker run -dti --network=host --name storage -e TRACKER_SERVER=11.11.11.11:22122 -v /var/fdfs/storage:/var/fdfs delron/fastdfs storage
```

# FastDFS客户端上传文件

Python版本的FastDFS客户端使用参考文档
https://github.com/jefforeilly/fdfs_client-py

- 1.安装FastDFS客户端扩展

```shell
cd到目录下--安装准备好的fdfs_client-py-master.zip到虚拟环境中
pip install fdfs_client-py-master.zip
pip install mutagen
pip install requests

```
- 2.准备FastDFS客户端扩展的配置文件

```1.utils.fastdfs.client.conf
拖拽 素材/client.conf 复制到fastdfs 
base_path=FastDFS客户端存放日志文件的目录
tracker_server=运行Tracker服务的机器ip:22122
3.FastDFS客户端demo实现文件存储
3.1 在 utils/fastdfs/demo.py
```

```python
# 1. 导入FastDFS客户端扩展
from fdfs_client.client import Fdfs_client

# 2. 创建FastDFS客户端实例
client = Fdfs_client('client.conf')

# 3. 调用FastDFS客户端上传文件方法
ret = client.upload_by_filename('/Users/lcf/Desktop/meizi.png')

```
- 输出结果
```
ret = {
'Group name': 'group1',
'Remote file_id': 'group1/M00/00/00/wKhnnlxw_gmAcoWmAAEXU5wmjPs35.jpeg',
'Status': 'Upload successed.',
'Local file name': '/Users/zhangjie/Desktop/kk.jpeg',
'Uploaded size': '69.00KB',
'Storage IP': '192.168.103.158'
 }
ret = {
'Group name': 'Storage组名',
'Remote file_id': '文件索引，可用于下载',
'Status': '文件上传结果反馈',
'Local file name': '上传文件全路径',
'Uploaded size': '文件大小',
'Storage IP': 'Storage地址'
 }
```

- 4. 浏览器下载并渲染图片--测试

```
思考：如何才能找到在Storage中存储的图片？

协议：
http
IP地址：192.168.103.158
Nginx服务器的IP地址。
因为 FastDFS 擅长存储静态文件，但是不擅长提供静态文件的下载服务，所以我们一般会将 Nginx 服务器绑定到 Storage ，提升下载性能。
端口：8888
Nginx服务器的端口。
路径：group1/M00/00/00/wKhnnlxw_gmAcoWmAAEXU5wmjPs35.jpeg
文件在Storage上的文件索引。
完整图片下载地址
http://192.168.103.158:8888/group1/M00/00/00/wKhnnlxw_gmAcoWmAAEXU5wmjPs35.jpeg
```